In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('../data/wine_cleared.csv')


In [2]:
wines = data.copy()
wines.head(1)

,Unnamed: 0,country,description,designation,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,35.363389,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia


In [3]:
wines['price_round'] = wines['price'].round().astype('int')

In [40]:
wines['year'] = wines['title'].str.findall('\d{4}').str.get(0)
# wines['year']

In [5]:
wines['is_usa'] = wines['country'].apply(lambda x: 1 if x == 'US' else 0)
wines['country'].value_counts(normalize=True).nlargest(3)

US        0.419558
France    0.170067
Italy     0.150414
Name: country, dtype: float64

In [6]:
wines['is_italy'] = wines['country'].apply(lambda x: 1 if x == 'Italy' else 0)
wines['is_france'] = wines['country'].apply(lambda x: 1 if x == 'France' else 0)
print(wines['is_france'].sum(), wines['is_italy'].sum())

22093 19540


In [7]:
wines['old_wine'] = wines['year'].apply(lambda x: 1 if x is not np.NaN and (int(x) < 2010) else 0)
wines['old_wine'].sum()

39781

In [8]:
wines['locality'] = wines['title'].str.findall('\((.+?)\)').str.get(0)


## Внешние источники данных


In [10]:
country_population = pd.read_csv("../data/country_population.csv", sep=';')
country_population

,country,population
0,China,"1,411,778,724"
1,India,"1,386,584,581"
2,US,"333,022,386"
3,Indonesia,"271,350,000"
4,Pakistan,"225,200,000"
...,...,...
236,Niue,"1,549"
237,Tokelau,"1,501"
238,Vatican City,825
239,Cocos Islands,573


In [11]:
country_population[country_population['country'] == 'Italy']

,country,population
24,Italy,"59,097,904"


In [18]:
wines = wines.join(country_population.set_index('country'), on='country')
print()
# country_population.set_index('country')

In [26]:
country_area = pd.read_csv("../data/country_area.csv", sep=';')
wines = wines.join(country_area.set_index('country'), on='country')
wines[wines['title'] == 'Gård 2014 Grand Klasse Reserve Lawrence Vineyards Viognier (Columbia Valley (WA))']['area']

94    9372610.0
Name: area, dtype: float64

## date-time


In [32]:
calls_list = [
    [460, '2013-12-17 04:55:39', '2013-12-17 04:55:44', '2013-12-17 04:55:45'],
    [12, '2013-12-16 20:03:20', '2013-12-16 20:03:22', '2013-12-16 20:07:13'],
    [56, '2013-12-16 20:03:20', '2013-12-16 20:03:20', '2013-12-16 20:05:04'],
    [980, '2013-12-16 20:03:20','2013-12-16 20:03:27', '2013-12-16 20:03:29'],
    [396, '2013-12-16 20:08:27', '2013-12-16 20:08:28','2013-12-16 20:12:03'],
    [449, '2013-12-16 20:03:20', '2013-12-16 20:03:25','2013-12-16 20:05:00'],
    [397, '2013-12-16 20:08:25', '2013-12-16 20:08:27', '2013-12-16 20:09:59'],
    [398, '2013-12-16 20:01:23', '2013-12-16 20:01:23', '2013-12-16 20:04:58'],
    [452, '2013-12-16 20:03:20', '2013-12-16 20:03:21','2013-12-16 20:04:55'],
    [440, '2013-12-16 20:03:20', '2013-12-16 20:04:26', '2013-12-16 20:04:32']
]

calls = pd.DataFrame(calls_list, columns=['client_id', 'agent_date', 'created_at', 'end_date'])
for col in calls.columns:
    if col == 'client_id': continue
    calls[col] = pd.to_datetime(calls[col])
calls['duration'] = (calls['end_date'] - calls['created_at']).dt.seconds
calls['time_connection'] = (calls['created_at'] - calls['agent_date']).dt.seconds
display(calls)
calls['time_connection'].sum()
    

,client_id,agent_date,created_at,end_date,duration,time_connection
0,460,2013-12-17 04:55:39,2013-12-17 04:55:44,2013-12-17 04:55:45,1,5
1,12,2013-12-16 20:03:20,2013-12-16 20:03:22,2013-12-16 20:07:13,231,2
2,56,2013-12-16 20:03:20,2013-12-16 20:03:20,2013-12-16 20:05:04,104,0
3,980,2013-12-16 20:03:20,2013-12-16 20:03:27,2013-12-16 20:03:29,2,7
4,396,2013-12-16 20:08:27,2013-12-16 20:08:28,2013-12-16 20:12:03,215,1
5,449,2013-12-16 20:03:20,2013-12-16 20:03:25,2013-12-16 20:05:00,95,5
6,397,2013-12-16 20:08:25,2013-12-16 20:08:27,2013-12-16 20:09:59,92,2
7,398,2013-12-16 20:01:23,2013-12-16 20:01:23,2013-12-16 20:04:58,215,0
8,452,2013-12-16 20:03:20,2013-12-16 20:03:21,2013-12-16 20:04:55,94,1
9,440,2013-12-16 20:03:20,2013-12-16 20:04:26,2013-12-16 20:04:32,6,66


89

In [34]:
calls['is_connection'] = calls['duration'].apply(lambda x: 1 if x > 10 else 0)
calls['is_connection'].sum()
# calls.loc[(calls['end_date'] - calls['created_at']).dt.seconds > 10, 'is_connection'] = 1


7

In [36]:
calls['time_diff'] = (calls['end_date'] - calls['agent_date']).dt.seconds
calls = calls.drop(columns=['agent_date', 'created_at' ,'end_date'], axis=1)
# calls['time_diff'].sum()
calls

,client_id,duration,time_connection,is_connection,time_diff
0,460,1,5,0,6
1,12,231,2,1,233
2,56,104,0,1,104
3,980,2,7,0,9
4,396,215,1,1,216
5,449,95,5,1,100
6,397,92,2,1,94
7,398,215,0,1,215
8,452,94,1,1,95
9,440,6,66,0,72


***

In [65]:
wines['year'] = pd.to_datetime(wines['year'], errors='coerce')
wines['year_diff'] = round((pd.to_datetime('2022-01-12') - wines['year']).dt.days)
wines['year_diff'].max()


98627.0